# Homework Session 11 Neural networks

### part 1

The purpose of the below is to classify days over years 2017-2018 by their corresponding mobility patterns between 10 zones in Taipei (quantified by an aggregated temporal network of subway ridership flows across the city)

In [49]:
!pip install keras
!pip install tensorflow

In [50]:
import numpy as np
import pandas as pd
import keras
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
from keras import backend as K
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
# from keras.optimizers import RMSprop
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import heapq

In [51]:
#read the data
TNet=pd.read_csv('taipeiD_TNet2.csv',header=None);

In [52]:
TNet.head() 
#each row represents a 10x10 adjacency matrix of the normalized Taipei subway mobility network between 10 zones flattened into a 100x1 row corresponding to a single day
#days start at jan-1-2017

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.017943,0.005415,0.003590,0.008316,0.007859,0.012942,0.012196,0.019543,0.001196,0.003327,...,0.002529,0.001533,0.001860,0.002375,0.005408,0.008922,0.003945,0.011075,0.005073,0.012708
1,0.021283,0.005215,0.003530,0.009359,0.007803,0.014288,0.011185,0.019044,0.001383,0.003499,...,0.002803,0.001757,0.001783,0.002549,0.005515,0.009650,0.003596,0.009618,0.005946,0.013709
2,0.028988,0.006511,0.005591,0.012970,0.007816,0.015878,0.010973,0.015768,0.002252,0.005388,...,0.004649,0.002555,0.002672,0.004291,0.007385,0.009558,0.004293,0.008791,0.010040,0.016301
3,0.029534,0.006471,0.005615,0.013017,0.007717,0.016098,0.011182,0.015815,0.002325,0.005443,...,0.004611,0.002473,0.002635,0.004195,0.007255,0.009487,0.004316,0.008729,0.010296,0.016437
4,0.029333,0.006525,0.005727,0.013098,0.007692,0.016358,0.011000,0.015677,0.002344,0.005527,...,0.004694,0.002515,0.002677,0.004222,0.007269,0.009921,0.004387,0.008923,0.010381,0.016914


In [53]:
#convert to an array and scale the data
X=np.array(TNet);

In [54]:
X=MinMaxScaler(feature_range=(0, 1), copy=True).fit_transform(X)

In [55]:
X.shape

(669, 100)

In [56]:
#define day of the week corresponding to each day of observation; 0-Sunday, 1-Monday,...,6-Saturday
y=np.array(range(669))%7; y[:10]

array([0, 1, 2, 3, 4, 5, 6, 0, 1, 2])

In [57]:
yc=np_utils.to_categorical(y) #get categorical binary variables isSunday, isMonday,...
yc[:5]

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

In [58]:
X_test=X[400:,:]; X_train=X[:400,:]; #split the data into training and test
y_test=yc[400:,:]; y_train=yc[:400,:]

In [59]:
# y_test

## Task 1. Classify weekdays/weekends
Label the rows with ones for weekends, zeros for weekdays.
Train a neural network with 4 layers of 30,10,3 and 1 (output) neurons over the training sample against this label, evaluating its performance over the test sample. Report the acheived accuracy (categorical) over the test sample

First three layers use relu activation function, last one - sigmoid.
Use loss='binary_crossentropy', optimizer='adam', 100 epochs, batch_size=20. 

In [60]:
y_test_1 = pd.DataFrame(y)
y_test_1 = y_test_1.rename(columns = {0:'ori'})
y_test_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669 entries, 0 to 668
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   ori     669 non-null    int64
dtypes: int64(1)
memory usage: 5.4 KB


In [61]:
y_test_1 = y_test_1.replace(0,10)
y_test_1

,ori
0,10
1,1
2,2
3,3
4,4
...,...
664,6
665,10
666,1
667,2


In [62]:
y_test_11 = y_test_1.replace(1,0)
y_test_12 = y_test_11.replace(2,0)
y_test_12 = y_test_12.replace(3,0)
y_test_12 = y_test_12.replace(4,0)
y_test_12 = y_test_12.replace(5,0)
y_test_12

,ori
0,10
1,0
2,0
3,0
4,0
...,...
664,6
665,10
666,0
667,0


In [63]:
y_test_1 = y_test_12.replace(10,1)
y_test_1 = y_test_1.replace(6,1)
y_test_1

,ori
0,1
1,0
2,0
3,0
4,0
...,...
664,1
665,1
666,0
667,0


In [64]:
X_test_1=X[400:,:]; X_train_1=X[:400,:]; #split the data into training and test
y_test_0=y_test_1[400:]; y_train_0=y_test_1[:400]

In [65]:
print(X_test_1.shape,y_test_0.shape,X_train_1.shape,y_train_0.shape)

(269, 100) (269, 1) (400, 100) (400, 1)


In [66]:
print(X_test.shape,y_test.shape,X_train.shape,y_train.shape)

(269, 100) (269, 7) (400, 100) (400, 7)


In [67]:
dim = X_train.shape[1]

In [68]:
np.random.seed(2019)
model = Sequential()
model.add(Dense(30, activation='relu', input_dim=dim))
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
model.fit(X_train_1, y_train_0, validation_data=(X_test_1, y_test_0), epochs=100, batch_size=20, verbose=2)
prob = model.predict(X_test_1)

Epoch 1/100
20/20 - 1s - loss: 0.6832 - categorical_accuracy: 1.0000 - val_loss: 0.6500 - val_categorical_accuracy: 1.0000 - 1s/epoch - 64ms/step
Epoch 2/100
20/20 - 0s - loss: 0.6249 - categorical_accuracy: 1.0000 - val_loss: 0.6145 - val_categorical_accuracy: 1.0000 - 83ms/epoch - 4ms/step
Epoch 3/100
20/20 - 0s - loss: 0.5942 - categorical_accuracy: 1.0000 - val_loss: 0.5885 - val_categorical_accuracy: 1.0000 - 78ms/epoch - 4ms/step
Epoch 4/100
20/20 - 0s - loss: 0.5716 - categorical_accuracy: 1.0000 - val_loss: 0.5707 - val_categorical_accuracy: 1.0000 - 67ms/epoch - 3ms/step
Epoch 5/100
20/20 - 0s - loss: 0.5553 - categorical_accuracy: 1.0000 - val_loss: 0.5558 - val_categorical_accuracy: 1.0000 - 142ms/epoch - 7ms/step
Epoch 6/100
20/20 - 0s - loss: 0.5446 - categorical_accuracy: 1.0000 - val_loss: 0.5434 - val_categorical_accuracy: 1.0000 - 167ms/epoch - 8ms/step
Epoch 7/100
20/20 - 0s - loss: 0.5324 - categorical_accuracy: 1.0000 - val_loss: 0.5349 - val_categorical_accuracy: 1

In [69]:
from sklearn.metrics import accuracy_score
print('Accuracy Rate: %f' % accuracy_score(y_test_0, prob.argmax(axis = 1)))

Accuracy Rate: 0.717472


## Task 2. Classify all days of the week
Train a neural network against the origial categorical label. Use 5 layers of 40,15,5 and 7 (outputs, representing probabilities for a current input to correspond to each of the weekdays) neurons over the training sample, evaluating its performance over the test sample (use 'categorical_accurary'). Report the acheived accuracy (categorical) over the test sample.

First three layers use relu activation function, last one - sigmoid.
Use loss='binary_crossentropy', optimizer='adam', 200 epochs, batch_size=20

In [70]:
np.random.seed(2019)
model = Sequential()
model.add(Dense(40, activation='relu', input_dim=dim))
model.add(Dense(15, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(7, activation='relu'))
model.add(Dense(7, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, batch_size=20, verbose=2)
prob1 = model.predict(X_test)

Epoch 1/200
20/20 - 2s - loss: 0.6539 - categorical_accuracy: 0.1325 - val_loss: 0.6347 - val_categorical_accuracy: 0.1561 - 2s/epoch - 80ms/step
Epoch 2/200
20/20 - 0s - loss: 0.6191 - categorical_accuracy: 0.1800 - val_loss: 0.6014 - val_categorical_accuracy: 0.2751 - 169ms/epoch - 8ms/step
Epoch 3/200
20/20 - 0s - loss: 0.5879 - categorical_accuracy: 0.2675 - val_loss: 0.5717 - val_categorical_accuracy: 0.2788 - 128ms/epoch - 6ms/step
Epoch 4/200
20/20 - 0s - loss: 0.5586 - categorical_accuracy: 0.2825 - val_loss: 0.5438 - val_categorical_accuracy: 0.2788 - 149ms/epoch - 7ms/step
Epoch 5/200
20/20 - 0s - loss: 0.5309 - categorical_accuracy: 0.2675 - val_loss: 0.5151 - val_categorical_accuracy: 0.2751 - 152ms/epoch - 8ms/step
Epoch 6/200
20/20 - 0s - loss: 0.5033 - categorical_accuracy: 0.2875 - val_loss: 0.4888 - val_categorical_accuracy: 0.2788 - 148ms/epoch - 7ms/step
Epoch 7/200
20/20 - 0s - loss: 0.4792 - categorical_accuracy: 0.2775 - val_loss: 0.4628 - val_categorical_accuracy

In [71]:
prob1_1=1.0*(prob1>0.5)

In [72]:
#Test accuracy
1.0*sum((y_test==1)==(prob1_1>0.5))/len(prob1_1)

array([0.98513011, 0.78810409, 0.85501859, 0.85501859, 0.84386617,
       0.95910781, 0.9739777 ])

## part 2

Use the same datasets as in the Advanced NN lab and further train NN with different architectures

## Task 1

Try the facial recognition task from NN lab 2 with different model specifications and report model accuracy:
1. Try different dropouts - 0.2, 0.3, 0.4
2. Try different convolution windows: (3$\times$3, 10$\times$10)

In [73]:
# load the data

from sklearn.datasets import fetch_lfw_people 
from keras.datasets import mnist
import warnings
warnings.simplefilter('ignore')
from sklearn.metrics import accuracy_score

lfw_people = fetch_lfw_people(min_faces_per_person = 70, resize = 0.4) 
  
# the images arrays to find the shapes (for plotting) 
n_samples, h, w = lfw_people.images.shape 
  
# Instead of providing 2D data, X has data already in the form  of a vector that 
# is required in this approach. 
X = lfw_people.data 
n_features = X.shape[1] 

y = lfw_people.target 
target_names = lfw_people.target_names 
n_classes = target_names.shape[0] 

In [74]:
# lets split and reform the dataset in the format compatible for CNN
# basically we are restructuring the data into grid shapes which are apt for convolutions and further steps

X_train, X_test, y_train, y_test = train_test_split( 
    X, y, test_size = 0.2)

X_train_pp = (X_train.reshape(X_train.shape[0], 50, 37, 1).astype('float32')) / 255 # normalize
X_test_pp = (X_test.reshape(X_test.shape[0], 50, 37, 1).astype('float32')) / 255

y_train_pp = np_utils.to_categorical(y_train)
y_test_pp = np_utils.to_categorical(y_test)

num_classes = y_train_pp.shape[1]

In [75]:
print(X_train_pp.shape, X_test_pp.shape)

(1030, 50, 37, 1) (258, 50, 37, 1)


In [76]:
# model dropouts - 0.2, convolution windows: 3 × 3

np.random.seed(2002)
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(50, 37, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), input_shape=(50, 37, 1), activation='relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
# should take under 2-3 minutes to train

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=50, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/50
6/6 - 3s - loss: 1.9581 - accuracy: 0.2883 - val_loss: 1.7411 - val_accuracy: 0.3992 - 3s/epoch - 530ms/step
Epoch 2/50
6/6 - 2s - loss: 1.6935 - accuracy: 0.4146 - val_loss: 1.6829 - val_accuracy: 0.3992 - 2s/epoch - 413ms/step
Epoch 3/50
6/6 - 2s - loss: 1.6525 - accuracy: 0.4146 - val_loss: 1.6642 - val_accuracy: 0.3992 - 2s/epoch - 414ms/step
Epoch 4/50
6/6 - 2s - loss: 1.6288 - accuracy: 0.4146 - val_loss: 1.6452 - val_accuracy: 0.3992 - 2s/epoch - 408ms/step
Epoch 5/50
6/6 - 2s - loss: 1.5859 - accuracy: 0.4155 - val_loss: 1.5801 - val_accuracy: 0.4109 - 2s/epoch - 416ms/step
Epoch 6/50
6/6 - 2s - loss: 1.5076 - accuracy: 0.4340 - val_loss: 1.4914 - val_accuracy: 0.4651 - 2s/epoch - 416ms/step
Epoch 7/50
6/6 - 2s - loss: 1.4271 - accuracy: 0.4932 - val_loss: 1.4256 - val_accuracy: 0.4690 - 2s/epoch - 414ms/step
Epoch 8/50
6/6 - 2s - loss: 1.3094 - accuracy: 0.5427 - val_loss: 1.6163 - val_accuracy: 0.4612 - 2s/epoch - 416ms/step
Epoch 9/50
6/6 - 2s - loss: 1.3613 - acc

In [77]:
# check accuracy score of predictions

predict_x=model.predict(X_test_pp) 
classes_x=np.argmax(predict_x,axis=1)
a2_1 = accuracy_score(y_test, classes_x)
a2_1

0.8565891472868217

In [78]:
# model dropouts - 0.2, convolution windows: 10 × 10

np.random.seed(2002)
model = Sequential()
model.add(Conv2D(32, (10,10), input_shape=(50, 37, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (10,10), input_shape=(50, 37, 1), activation='relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
# should take under 2-3 minutes to train

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=50, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/50
6/6 - 4s - loss: 1.9344 - accuracy: 0.2563 - val_loss: 1.8281 - val_accuracy: 0.3992 - 4s/epoch - 737ms/step
Epoch 2/50
6/6 - 4s - loss: 1.7426 - accuracy: 0.4146 - val_loss: 1.7228 - val_accuracy: 0.3992 - 4s/epoch - 619ms/step
Epoch 3/50
6/6 - 4s - loss: 1.6769 - accuracy: 0.4146 - val_loss: 1.6980 - val_accuracy: 0.3992 - 4s/epoch - 621ms/step
Epoch 4/50
6/6 - 4s - loss: 1.6831 - accuracy: 0.4146 - val_loss: 1.6962 - val_accuracy: 0.3992 - 4s/epoch - 612ms/step
Epoch 5/50
6/6 - 4s - loss: 1.6812 - accuracy: 0.4146 - val_loss: 1.7010 - val_accuracy: 0.3992 - 4s/epoch - 614ms/step
Epoch 6/50
6/6 - 4s - loss: 1.6730 - accuracy: 0.4146 - val_loss: 1.7197 - val_accuracy: 0.3992 - 4s/epoch - 610ms/step
Epoch 7/50
6/6 - 4s - loss: 1.6749 - accuracy: 0.4146 - val_loss: 1.6949 - val_accuracy: 0.3992 - 4s/epoch - 611ms/step
Epoch 8/50
6/6 - 4s - loss: 1.6685 - accuracy: 0.4146 - val_loss: 1.6975 - val_accuracy: 0.3992 - 4s/epoch - 611ms/step
Epoch 9/50
6/6 - 4s - loss: 1.6649 - acc

In [79]:
# check accuracy score of predictions

predict_x=model.predict(X_test_pp) 
classes_x=np.argmax(predict_x,axis=1)
a2_2 = accuracy_score(y_test, classes_x)
a2_2

0.7403100775193798

In [80]:
# model dropouts - 0.3, convolution windows: 3 × 3

np.random.seed(2002)
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(50, 37, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), input_shape=(50, 37, 1), activation='relu'))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
# should take under 2-3 minutes to train

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=50, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/50
6/6 - 6s - loss: 1.8513 - accuracy: 0.3437 - val_loss: 1.6998 - val_accuracy: 0.3992 - 6s/epoch - 1s/step
Epoch 2/50
6/6 - 3s - loss: 1.6797 - accuracy: 0.4146 - val_loss: 1.6940 - val_accuracy: 0.3992 - 3s/epoch - 435ms/step
Epoch 3/50
6/6 - 2s - loss: 1.6636 - accuracy: 0.4146 - val_loss: 1.6787 - val_accuracy: 0.3992 - 2s/epoch - 416ms/step
Epoch 4/50
6/6 - 3s - loss: 1.6539 - accuracy: 0.4146 - val_loss: 1.6619 - val_accuracy: 0.3992 - 3s/epoch - 421ms/step
Epoch 5/50
6/6 - 2s - loss: 1.6235 - accuracy: 0.4146 - val_loss: 1.6333 - val_accuracy: 0.3992 - 2s/epoch - 413ms/step
Epoch 6/50
6/6 - 2s - loss: 1.5823 - accuracy: 0.4146 - val_loss: 1.5865 - val_accuracy: 0.3992 - 2s/epoch - 412ms/step
Epoch 7/50
6/6 - 3s - loss: 1.5238 - accuracy: 0.4350 - val_loss: 1.5110 - val_accuracy: 0.4574 - 3s/epoch - 425ms/step
Epoch 8/50
6/6 - 2s - loss: 1.4494 - accuracy: 0.4757 - val_loss: 1.4409 - val_accuracy: 0.5000 - 2s/epoch - 412ms/step
Epoch 9/50
6/6 - 2s - loss: 1.3462 - accura

In [81]:
# check accuracy score of predictions

predict_x=model.predict(X_test_pp) 
classes_x=np.argmax(predict_x,axis=1)
a3_1 = accuracy_score(y_test, classes_x)
a3_1

0.8643410852713178

In [82]:
# model dropouts - 0.3, convolution windows: 10 × 10

np.random.seed(2002)
model = Sequential()
model.add(Conv2D(32, (10,10), input_shape=(50, 37, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (10,10), input_shape=(50, 37, 1), activation='relu'))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
# should take under 2-3 minutes to train

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=50, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/50
6/6 - 4s - loss: 1.8223 - accuracy: 0.3010 - val_loss: 1.7060 - val_accuracy: 0.3992 - 4s/epoch - 734ms/step
Epoch 2/50
6/6 - 4s - loss: 1.6926 - accuracy: 0.4146 - val_loss: 1.6864 - val_accuracy: 0.3992 - 4s/epoch - 616ms/step
Epoch 3/50
6/6 - 4s - loss: 1.6741 - accuracy: 0.4146 - val_loss: 1.6898 - val_accuracy: 0.3992 - 4s/epoch - 626ms/step
Epoch 4/50
6/6 - 4s - loss: 1.6725 - accuracy: 0.4146 - val_loss: 1.7670 - val_accuracy: 0.3992 - 4s/epoch - 616ms/step
Epoch 5/50
Epoch 6/50
6/6 - 4s - loss: 1.6827 - accuracy: 0.4146 - val_loss: 1.6893 - val_accuracy: 0.3992 - 4s/epoch - 614ms/step
Epoch 7/50
6/6 - 4s - loss: 1.6672 - accuracy: 0.4146 - val_loss: 1.6767 - val_accuracy: 0.3992 - 4s/epoch - 616ms/step
Epoch 8/50
6/6 - 4s - loss: 1.6567 - accuracy: 0.4146 - val_loss: 1.6598 - val_accuracy: 0.3992 - 4s/epoch - 617ms/step
Epoch 9/50
6/6 - 4s - loss: 1.6389 - accuracy: 0.4146 - val_loss: 1.6414 - val_accuracy: 0.3992 - 4s/epoch - 622ms/step
Epoch 10/50
6/6 - 4s - loss: 

In [83]:
# check accuracy score of predictions

predict_x=model.predict(X_test_pp) 
classes_x=np.argmax(predict_x,axis=1)
a3_2 = accuracy_score(y_test, classes_x)
a3_2

0.7906976744186046

In [84]:
# model dropouts - 0.4, convolution windows: 3 × 3

np.random.seed(2002)
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(50, 37, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), input_shape=(50, 37, 1), activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
# should take under 2-3 minutes to train

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=50, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/50
6/6 - 3s - loss: 1.9771 - accuracy: 0.2874 - val_loss: 1.7908 - val_accuracy: 0.4341 - 3s/epoch - 528ms/step
Epoch 2/50
6/6 - 2s - loss: 1.7408 - accuracy: 0.4068 - val_loss: 1.7068 - val_accuracy: 0.3992 - 2s/epoch - 407ms/step
Epoch 3/50
6/6 - 2s - loss: 1.6817 - accuracy: 0.4146 - val_loss: 1.6873 - val_accuracy: 0.3992 - 2s/epoch - 414ms/step
Epoch 4/50
6/6 - 2s - loss: 1.6657 - accuracy: 0.4146 - val_loss: 1.6715 - val_accuracy: 0.3992 - 2s/epoch - 408ms/step
Epoch 5/50
6/6 - 2s - loss: 1.6299 - accuracy: 0.4146 - val_loss: 1.6350 - val_accuracy: 0.3992 - 2s/epoch - 408ms/step
Epoch 6/50
6/6 - 2s - loss: 1.5916 - accuracy: 0.4155 - val_loss: 1.5886 - val_accuracy: 0.3992 - 2s/epoch - 410ms/step
Epoch 7/50
6/6 - 2s - loss: 1.5334 - accuracy: 0.4184 - val_loss: 1.5214 - val_accuracy: 0.4806 - 2s/epoch - 412ms/step
Epoch 8/50
6/6 - 3s - loss: 1.4487 - accuracy: 0.4893 - val_loss: 1.4663 - val_accuracy: 0.5116 - 3s/epoch - 422ms/step
Epoch 9/50
6/6 - 3s - loss: 1.3860 - acc

In [85]:
# check accuracy score of predictions

predict_x=model.predict(X_test_pp) 
classes_x=np.argmax(predict_x,axis=1)
a4_1 = accuracy_score(y_test, classes_x)
a4_1

0.8604651162790697

In [89]:
# model dropouts - 0.4, convolution windows: 10 × 10

np.random.seed(2002)
model = Sequential()
model.add(Conv2D(32, (10,10), input_shape=(50, 37, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (10,10), input_shape=(50, 37, 1), activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
# should take under 2-3 minutes to train

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=50, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/50
6/6 - 8s - loss: 1.7582 - accuracy: 0.3553 - val_loss: 1.6982 - val_accuracy: 0.3992 - 8s/epoch - 1s/step
Epoch 2/50
6/6 - 4s - loss: 1.6766 - accuracy: 0.4146 - val_loss: 1.7189 - val_accuracy: 0.3992 - 4s/epoch - 655ms/step
Epoch 3/50
6/6 - 4s - loss: 1.6664 - accuracy: 0.4146 - val_loss: 1.6689 - val_accuracy: 0.3992 - 4s/epoch - 613ms/step
Epoch 4/50
6/6 - 4s - loss: 1.6333 - accuracy: 0.4146 - val_loss: 1.6194 - val_accuracy: 0.4109 - 4s/epoch - 609ms/step
Epoch 5/50
6/6 - 4s - loss: 1.5778 - accuracy: 0.4282 - val_loss: 1.5921 - val_accuracy: 0.4767 - 4s/epoch - 615ms/step
Epoch 6/50
6/6 - 4s - loss: 1.5367 - accuracy: 0.4505 - val_loss: 1.5318 - val_accuracy: 0.4651 - 4s/epoch - 614ms/step
Epoch 7/50
6/6 - 4s - loss: 1.4525 - accuracy: 0.4825 - val_loss: 1.4718 - val_accuracy: 0.4884 - 4s/epoch - 613ms/step
Epoch 8/50
6/6 - 4s - loss: 1.4293 - accuracy: 0.4903 - val_loss: 1.4276 - val_accuracy: 0.5504 - 4s/epoch - 615ms/step
Epoch 9/50
6/6 - 4s - loss: 1.3305 - accura

In [90]:
# check accuracy score of predictions

predict_x=model.predict(X_test_pp) 
classes_x=np.argmax(predict_x,axis=1)
a4_2 = accuracy_score(y_test, classes_x)
a4_2

0.8565891472868217

In [91]:
print("The accuracy of the model with dropout 0.2 and convolution windows 3x3 is :{}".format(a2_1))
print("The accuracy of the model with dropout 0.2 and convolution windows 10x10 is :{}".format(a2_2))
print("The accuracy of the model with dropout 0.3 and convolution windows 3x3 is :{}".format(a3_1))
print("The accuracy of the model with dropout 0.3 and convolution windows 10x10 is :{}".format(a3_2))
print("The accuracy of the model with dropout 0.4 and convolution windows 3x3 is :{}".format(a4_1))
print("The accuracy of the model with dropout 0.4 and convolution windows 10x10 is :{}".format(a4_2))

The accuracy of the model with dropout 0.2 and convolution windows 3x3 is :0.8565891472868217
The accuracy of the model with dropout 0.2 and convolution windows 10x10 is :0.7403100775193798
The accuracy of the model with dropout 0.3 and convolution windows 3x3 is :0.8643410852713178
The accuracy of the model with dropout 0.3 and convolution windows 10x10 is :0.7906976744186046
The accuracy of the model with dropout 0.4 and convolution windows 3x3 is :0.8604651162790697
The accuracy of the model with dropout 0.4 and convolution windows 10x10 is :0.8565891472868217
